In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 

import datetime
from datetime import timedelta
from timeit import default_timer as timer

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from tf_netbuilder.files import download_checkpoint
from tf_netbuilder_ext.extensions import register_tf_netbuilder_extensions
from models import create_openpose_singlenet
from dataset.generators import get_dataset
from util import plot_to_image, probe_model_singlenet

print("TF:", tf.__version__)
print("Cuda:", tf.test.is_built_with_cuda())
print("GPU:", tf.test.is_gpu_available())

TF: 2.5.0
Cuda: True
GPU: True


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
'''
tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=7168)]
)
'''
pretrained_mobilenet_v3_url = "https://github.com/michalfaber/tf_netbuilder/releases/download/v1.0/mobilenet_v3_224_1_0.zip"
annot_path_train = '../datasets/coco_2017_dataset/annotations/person_keypoints_train2017.json'
img_dir_train = '../datasets/coco_2017_dataset/train2017/'
annot_path_val = '../datasets/coco_2017_dataset/annotations/person_keypoints_val2017.json'
img_dir_val = '../datasets/coco_2017_dataset/val2017/'
checkpoints_folder = './tf_ckpts_singlenet/0629_epoch300'
output_weights = 'output_singlenet/0629_epoch300/openpose_singlenet'

batch_size = 10
lr = 2.5e-5
max_epochs = 300

In [3]:
def eucl_loss(y_true, y_pred):
    return tf.reduce_sum(tf.math.squared_difference(y_pred, y_true))

@tf.function
def train_one_step(model, optimizer, x, y_true):
    with tf.GradientTape() as tape:
        y_pred = model(x)

        losses = [eucl_loss(y_true[0], y_pred[0]),
                  eucl_loss(y_true[0], y_pred[1]),
                  eucl_loss(y_true[0], y_pred[2]),
                  eucl_loss(y_true[1], y_pred[3])
                  ]

        total_loss = tf.reduce_sum(losses)

    grads = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return losses, total_loss

def train(ds_train, ds_val, model, optimizer, ckpt, last_epoch, last_step, max_epochs, steps_per_epoch):
    train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
    train_loss_heatmap = tf.keras.metrics.Mean('train_loss_heatmap', dtype=tf.float32)
    train_loss_paf = tf.keras.metrics.Mean('train_loss_paf', dtype=tf.float32)

    val_loss = tf.keras.metrics.Mean('val_loss', dtype=tf.float32)
    val_loss_heatmap = tf.keras.metrics.Mean('val_loss_heatmap', dtype=tf.float32)
    val_loss_paf = tf.keras.metrics.Mean('val_loss_paf', dtype=tf.float32)

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs_singlenet/gradient_tape/' + current_time + '/train'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    val_log_dir = 'logs_singlenet/gradient_tape/' + current_time + '/val'
    val_summary_writer = tf.summary.create_file_writer(val_log_dir)

    output_paf_idx = 2
    output_heatmap_idx = 3

    # determine start epoch in case the training has been stopped manually and resumed
    resume = last_step != 0 and (steps_per_epoch - last_step) != 0
    if resume:
        start_epoch = last_epoch
    else:
        start_epoch = last_epoch + 1

    # start processing
    for epoch in range(start_epoch, max_epochs + 1, 1):

        start = timer()
        print("Start processing epoch {}".format(epoch))

        # set the initial step index depending on if you resumed the processing
        if resume:
            step = last_step + 1
            data_iter = ds_train.skip(last_step)
            print(f"Skipping {last_step} steps (May take a few minutes)...")
            resume = False
        else:
            step = 0
            data_iter = ds_train

        # process steps
        for x, y in data_iter:
            step += 1

            losses, total_loss = train_one_step(model, optimizer, x, y)

            train_loss(total_loss)
            train_loss_heatmap(losses[output_heatmap_idx])
            train_loss_paf(losses[output_paf_idx])

            if step % 10 == 0:
                tf.print('Epoch', epoch, f'Step {step}/{steps_per_epoch}', 'Paf1', losses[0], 'Paf2', losses[1], 'Paf3', losses[2],
                         'Heatmap', losses[3], 'Total loss', total_loss)

                with train_summary_writer.as_default():
                    summary_step = (epoch - 1) * steps_per_epoch + step - 1
                    tf.summary.scalar('loss', train_loss.result(), step=summary_step)
                    tf.summary.scalar('loss_heatmap', train_loss_heatmap.result(), step=summary_step)
                    tf.summary.scalar('loss_paf', train_loss_paf.result(), step=summary_step)

            if step % 100 == 0:
                figure = probe_model_singlenet(model, test_img_path="resources/ski_224.jpg")
                with train_summary_writer.as_default():
                    tf.summary.image("Test prediction", plot_to_image(figure), step=step)

            if step % 1000 == 0:
                ckpt.step.assign(step)
                ckpt.epoch.assign(epoch)
                save_path = manager.save()
                print("Saved checkpoint for step {}: {}".format(step, save_path))

            if step >= steps_per_epoch:
                break

        print("Completed epoch {}. Saving weights...".format(epoch))
        model.save_weights(output_weights, overwrite=True)

        # save checkpoint at the end of an epoch
        ckpt.step.assign(step)
        ckpt.epoch.assign(epoch)
        manager.save()

        # reset metrics every epoch
        train_loss.reset_states()
        train_loss_heatmap.reset_states()
        train_loss_paf.reset_states()

        end = timer()

        print("Epoch training time: " + str(timedelta(seconds=end - start)))

        # calculate validation loss
        print("Calculating validation losses...")
        for val_step, (x_val, y_val_true) in enumerate(ds_val):

            if val_step % 1000 == 0:
                print(f"Validation step {val_step} ...")

            y_val_pred = model(x_val)
            losses = [eucl_loss(y_val_true[0], y_val_pred[0]),
                      eucl_loss(y_val_true[0], y_val_pred[1]),
                      eucl_loss(y_val_true[0], y_val_pred[2]),
                      eucl_loss(y_val_true[1], y_val_pred[3])
                      ]

            total_loss = tf.reduce_sum(losses)
            val_loss(total_loss)
            val_loss_heatmap(losses[output_heatmap_idx])
            val_loss_paf(losses[output_paf_idx])

        val_loss_res = val_loss.result()
        val_loss_heatmap_res = val_loss_heatmap.result()
        val_loss_paf_res = val_loss_paf.result()

        print(f'Validation losses for epoch: {epoch} : Loss paf {val_loss_paf_res}, Loss heatmap '
              f'{val_loss_heatmap_res}, Total loss {val_loss_res}')

        with val_summary_writer.as_default():
            tf.summary.scalar('val_loss', val_loss_res, step=epoch)
            tf.summary.scalar('val_loss_heatmap', val_loss_heatmap_res, step=epoch)
            tf.summary.scalar('val_loss_paf', val_loss_paf_res, step=epoch)

        val_loss.reset_states()
        val_loss_heatmap.reset_states()
        val_loss_paf.reset_states()

In [4]:
if __name__ == '__main__':

    # registering custom blocks types
    register_tf_netbuilder_extensions()

    # loading datasets
    ds_train, ds_train_size = get_dataset(annot_path_train, img_dir_train, batch_size)
    ds_val, ds_val_size = get_dataset(annot_path_val, img_dir_val, batch_size, strict=True)
    print(f"Training samples: {ds_train_size} , Validation samples: {ds_val_size}")

    steps_per_epoch = ds_train_size // batch_size
    # steps_per_epoch_val = ds_val_size // batch_size

    # creating model, optimizers etc
    model = create_openpose_singlenet(pretrained=False)
    optimizer = Adam(lr)

    # loading previous state if required
    ckpt = tf.train.Checkpoint(step=tf.Variable(0), epoch=tf.Variable(0), optimizer=optimizer, net=model)
    manager = tf.train.CheckpointManager(ckpt, checkpoints_folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    last_step = int(ckpt.step)
    last_epoch = int(ckpt.epoch)

    if manager.latest_checkpoint:
        print(f"Restored from {manager.latest_checkpoint}")
        print(f"Resumed from epoch {last_epoch}, step {last_step}")
    else:
        print("Initializing from scratch.")

        path = download_checkpoint(pretrained_mobilenet_v3_url)
        model.load_weights(path).expect_partial()

    train(ds_train, ds_val, model, optimizer, ckpt, last_epoch, last_step, max_epochs, steps_per_epoch)
    print("Done!")

loading annotations into memory...
Done (t=4.91s)
creating index...
index created!
Loading image annot 0/118287
Loading image annot 1000/118287
Loading image annot 8000/118287
Loading image annot 10000/118287
Loading image annot 11000/118287
Loading image annot 12000/118287
Loading image annot 14000/118287
Loading image annot 16000/118287
Loading image annot 19000/118287
Loading image annot 22000/118287
Loading image annot 24000/118287
Loading image annot 31000/118287
Loading image annot 37000/118287
Loading image annot 38000/118287
Loading image annot 40000/118287
Loading image annot 47000/118287
Loading image annot 60000/118287
Loading image annot 63000/118287
Loading image annot 65000/118287
Loading image annot 66000/118287
Loading image annot 69000/118287
Loading image annot 72000/118287
Loading image annot 73000/118287
Loading image annot 75000/118287
Loading image annot 76000/118287
Loading image annot 80000/118287
Loading image annot 82000/118287
Loading image annot 84000/118287

Epoch 295 Step 550/5304 Paf1 446.759155 Paf2 423.652954 Paf3 409.639832 Heatmap 121.427948 Total loss 1401.47986
Epoch 295 Step 560/5304 Paf1 925.836731 Paf2 883.80011 Paf3 885.547302 Heatmap 270.64151 Total loss 2965.82568
Epoch 295 Step 570/5304 Paf1 921.525146 Paf2 883.07196 Paf3 858.827881 Heatmap 248.493332 Total loss 2911.91846
Epoch 295 Step 580/5304 Paf1 657.262878 Paf2 614.529297 Paf3 606.675171 Heatmap 214.955688 Total loss 2093.4231
Epoch 295 Step 590/5304 Paf1 548.160645 Paf2 490.531616 Paf3 500.524719 Heatmap 142.324768 Total loss 1681.54175
Epoch 295 Step 600/5304 Paf1 785.998291 Paf2 736.568 Paf3 730.578552 Heatmap 270.496155 Total loss 2523.64111
Epoch 295 Step 610/5304 Paf1 556.391968 Paf2 533.737366 Paf3 524.51123 Heatmap 161.930298 Total loss 1776.57092
Epoch 295 Step 620/5304 Paf1 427.12381 Paf2 378.607269 Paf3 386.979279 Heatmap 134.5495 Total loss 1327.25989
Epoch 295 Step 630/5304 Paf1 708.95575 Paf2 651.543823 Paf3 658.207703 Heatmap 190.674042 Total loss 2209.3

Epoch 295 Step 1280/5304 Paf1 627.852966 Paf2 568.160889 Paf3 580.300903 Heatmap 191.72908 Total loss 1968.04395
Epoch 295 Step 1290/5304 Paf1 417.925415 Paf2 384.723846 Paf3 392.700104 Heatmap 131.663391 Total loss 1327.01282
Epoch 295 Step 1300/5304 Paf1 695.310913 Paf2 654.325623 Paf3 656.633179 Heatmap 244.514465 Total loss 2250.78418
Epoch 295 Step 1310/5304 Paf1 800.478394 Paf2 750.641174 Paf3 755.150391 Heatmap 233.60466 Total loss 2539.87476
Epoch 295 Step 1320/5304 Paf1 675.138184 Paf2 630.806458 Paf3 642.573181 Heatmap 217.0224 Total loss 2165.54
Epoch 295 Step 1330/5304 Paf1 539.206299 Paf2 503.906891 Paf3 509.716461 Heatmap 147.831177 Total loss 1700.66077
Epoch 295 Step 1340/5304 Paf1 1008.76123 Paf2 958.660034 Paf3 960.474731 Heatmap 344.365906 Total loss 3272.26172
Epoch 295 Step 1350/5304 Paf1 813.461853 Paf2 771.471252 Paf3 767.496887 Heatmap 261.676941 Total loss 2614.10693
Epoch 295 Step 1360/5304 Paf1 606.033142 Paf2 574.422058 Paf3 561.388123 Heatmap 181.484818 Tot

Saved checkpoint for step 2000: ./tf_ckpts_singlenet/0629_epoch300/ckpt-1766
Epoch 295 Step 2010/5304 Paf1 791.403625 Paf2 766.485901 Paf3 759.455933 Heatmap 219.10202 Total loss 2536.44751
Epoch 295 Step 2020/5304 Paf1 707.369385 Paf2 649.283813 Paf3 638.043152 Heatmap 204.66745 Total loss 2199.36377
Epoch 295 Step 2030/5304 Paf1 761.681824 Paf2 722.978638 Paf3 712.382568 Heatmap 192.095184 Total loss 2389.13818
Epoch 295 Step 2040/5304 Paf1 467.562286 Paf2 414.197815 Paf3 414.031342 Heatmap 120.751526 Total loss 1416.54297
Epoch 295 Step 2050/5304 Paf1 450.952728 Paf2 430.147339 Paf3 433.926788 Heatmap 190.439911 Total loss 1505.4668
Epoch 295 Step 2060/5304 Paf1 862.188904 Paf2 804.651611 Paf3 829.193665 Heatmap 235.760468 Total loss 2731.79468
Epoch 295 Step 2070/5304 Paf1 772.975342 Paf2 703.136353 Paf3 711.305481 Heatmap 188.363434 Total loss 2375.78076
Epoch 295 Step 2080/5304 Paf1 605.318726 Paf2 565.522461 Paf3 570.231812 Heatmap 197.195892 Total loss 1938.26892
Epoch 295 Step

Epoch 295 Step 2730/5304 Paf1 855.588623 Paf2 800.630127 Paf3 798.685303 Heatmap 275.414398 Total loss 2730.31836
Epoch 295 Step 2740/5304 Paf1 644.54187 Paf2 634.671936 Paf3 635.685791 Heatmap 202.56041 Total loss 2117.46
Epoch 295 Step 2750/5304 Paf1 711.183105 Paf2 673.064392 Paf3 678.927612 Heatmap 250.301117 Total loss 2313.47632
Epoch 295 Step 2760/5304 Paf1 643.632507 Paf2 617.979797 Paf3 598.504639 Heatmap 195.480469 Total loss 2055.59741
Epoch 295 Step 2770/5304 Paf1 683.392334 Paf2 619.169189 Paf3 623.969116 Heatmap 224.147659 Total loss 2150.67822
Epoch 295 Step 2780/5304 Paf1 539.723267 Paf2 490.768555 Paf3 487.119 Heatmap 156.859207 Total loss 1674.47
Epoch 295 Step 2790/5304 Paf1 377.486023 Paf2 351.947144 Paf3 356.190155 Heatmap 122.728683 Total loss 1208.35205
Epoch 295 Step 2800/5304 Paf1 755.990601 Paf2 710.947632 Paf3 697.455261 Heatmap 243.030548 Total loss 2407.42407
Epoch 295 Step 2810/5304 Paf1 850.638855 Paf2 807.922 Paf3 813.088501 Heatmap 304.736969 Total loss

...

Epoch 300 Step 2220/5304 Paf1 854.885925 Paf2 843.037231 Paf3 845.555664 Heatmap 250.104614 Total loss 2793.5835
Epoch 300 Step 2230/5304 Paf1 632.508301 Paf2 607.200745 Paf3 608.652 Heatmap 161.166977 Total loss 2009.52795
Epoch 300 Step 2240/5304 Paf1 639.206299 Paf2 611.589111 Paf3 602.768311 Heatmap 165.110077 Total loss 2018.67383
Epoch 300 Step 2250/5304 Paf1 818.838684 Paf2 771.856079 Paf3 783.602417 Heatmap 264.492432 Total loss 2638.78955
Epoch 300 Step 2260/5304 Paf1 646.628 Paf2 603.310242 Paf3 626.097778 Heatmap 193.013641 Total loss 2069.0498
Epoch 300 Step 2270/5304 Paf1 774.180969 Paf2 714.332764 Paf3 726.553833 Heatmap 264.938965 Total loss 2480.00635
Epoch 300 Step 2280/5304 Paf1 890.679077 Paf2 836.674622 Paf3 838.78363 Heatmap 292.04068 Total loss 2858.17822
Epoch 300 Step 2290/5304 Paf1 952.024231 Paf2 892.921936 Paf3 897.112793 Heatmap 273.319122 Total loss 3015.37793
Epoch 300 Step 2300/5304 Paf1 513.031616 Paf2 496.665405 Paf3 495.210419 Heatmap 156.274963 Total 

Epoch 300 Step 2950/5304 Paf1 672.858765 Paf2 651.500366 Paf3 650.704163 Heatmap 244.141266 Total loss 2219.20459
Epoch 300 Step 2960/5304 Paf1 788.179688 Paf2 728.44989 Paf3 726.145813 Heatmap 251.521713 Total loss 2494.29712
Epoch 300 Step 2970/5304 Paf1 550.433 Paf2 510.263855 Paf3 493.07608 Heatmap 151.33783 Total loss 1705.11072
Epoch 300 Step 2980/5304 Paf1 711.778 Paf2 684.02655 Paf3 657.220032 Heatmap 181.383316 Total loss 2234.40796
Epoch 300 Step 2990/5304 Paf1 695.574 Paf2 685.439758 Paf3 682.006226 Heatmap 224.562378 Total loss 2287.58228
Epoch 300 Step 3000/5304 Paf1 538.978394 Paf2 488.60437 Paf3 493.18042 Heatmap 155.013763 Total loss 1675.77698
Saved checkpoint for step 3000: ./tf_ckpts_singlenet/0629_epoch300/ckpt-1797
Epoch 300 Step 3010/5304 Paf1 465.6185 Paf2 425.193115 Paf3 437.584595 Heatmap 129.937195 Total loss 1458.3335
Epoch 300 Step 3020/5304 Paf1 671.998535 Paf2 603.172424 Paf3 622.85675 Heatmap 211.95697 Total loss 2109.98462
Epoch 300 Step 3030/5304 Paf1 6

Epoch 300 Step 3670/5304 Paf1 732.708069 Paf2 660.232666 Paf3 645.816284 Heatmap 228.208298 Total loss 2266.96533
Epoch 300 Step 3680/5304 Paf1 713.24646 Paf2 673.137085 Paf3 682.014 Heatmap 190.455017 Total loss 2258.85254
Epoch 300 Step 3690/5304 Paf1 594.954041 Paf2 576.927368 Paf3 562.063477 Heatmap 209.327042 Total loss 1943.27185
Epoch 300 Step 3700/5304 Paf1 664.645142 Paf2 622.184082 Paf3 639.88562 Heatmap 265.028 Total loss 2191.74292
Epoch 300 Step 3710/5304 Paf1 580.507751 Paf2 544.388184 Paf3 538.072144 Heatmap 166.786316 Total loss 1829.75439
Epoch 300 Step 3720/5304 Paf1 707.27594 Paf2 687.502319 Paf3 674.938904 Heatmap 219.974701 Total loss 2289.69189
Epoch 300 Step 3730/5304 Paf1 616.749268 Paf2 566.730103 Paf3 564.811279 Heatmap 163.572662 Total loss 1911.86328
Epoch 300 Step 3740/5304 Paf1 651.635681 Paf2 613.514465 Paf3 611.333862 Heatmap 163.31488 Total loss 2039.79883
Epoch 300 Step 3750/5304 Paf1 711.706299 Paf2 642.765686 Paf3 642.770264 Heatmap 216.172516 Total 

Epoch 300 Step 4390/5304 Paf1 532.071716 Paf2 467.874878 Paf3 475.104767 Heatmap 150.421631 Total loss 1625.4729
Epoch 300 Step 4400/5304 Paf1 622.494 Paf2 590.843 Paf3 579.306 Heatmap 215.489273 Total loss 2008.13232
Epoch 300 Step 4410/5304 Paf1 687.524292 Paf2 646.050171 Paf3 650.748413 Heatmap 233.536102 Total loss 2217.85889
Epoch 300 Step 4420/5304 Paf1 711.301392 Paf2 645.421875 Paf3 623.460754 Heatmap 198.606918 Total loss 2178.79102
Epoch 300 Step 4430/5304 Paf1 885.177734 Paf2 775.789307 Paf3 793.52 Heatmap 264.199371 Total loss 2718.68652
Epoch 300 Step 4440/5304 Paf1 1027.57434 Paf2 977.55835 Paf3 987.681213 Heatmap 352.846039 Total loss 3345.66
Epoch 300 Step 4450/5304 Paf1 684.134521 Paf2 639.516113 Paf3 637.383 Heatmap 225.969543 Total loss 2187.00317
Epoch 300 Step 4460/5304 Paf1 585.862549 Paf2 548.871094 Paf3 541.323853 Heatmap 165.27359 Total loss 1841.33105
Epoch 300 Step 4470/5304 Paf1 524.549561 Paf2 468.238495 Paf3 484.055511 Heatmap 128.224945 Total loss 1605.06

Epoch 300 Step 5120/5304 Paf1 749.65 Paf2 714.565247 Paf3 715.611938 Heatmap 243.297821 Total loss 2423.125
Epoch 300 Step 5130/5304 Paf1 384.43988 Paf2 356.984344 Paf3 367.221191 Heatmap 125.634773 Total loss 1234.28015
Epoch 300 Step 5140/5304 Paf1 668.983948 Paf2 632.555542 Paf3 617.728 Heatmap 210.139496 Total loss 2129.40723
Epoch 300 Step 5150/5304 Paf1 797.634033 Paf2 741.452881 Paf3 764.952576 Heatmap 251.396362 Total loss 2555.43579
Epoch 300 Step 5160/5304 Paf1 611.222107 Paf2 562.444092 Paf3 552.874756 Heatmap 199.497986 Total loss 1926.03906
Epoch 300 Step 5170/5304 Paf1 542.624512 Paf2 501.748535 Paf3 511.456757 Heatmap 153.903946 Total loss 1709.73376
Epoch 300 Step 5180/5304 Paf1 865.142944 Paf2 797.945251 Paf3 807.537 Heatmap 265.729187 Total loss 2736.35425
Epoch 300 Step 5190/5304 Paf1 703.665527 Paf2 653.988098 Paf3 654.364258 Heatmap 234.859406 Total loss 2246.8772
Epoch 300 Step 5200/5304 Paf1 566.988342 Paf2 535.429443 Paf3 542.869446 Heatmap 156.031174 Total loss